## 加载数据

In [1]:
# from utils.file_utils import load_base_config

# config = load_base_config("conf/config.yaml")
data_file = "tang/tang.txt"

In [2]:
# 读取数据
with open(data_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# 去除空行和多余的空白符
chinese_data = [line.strip() for line in lines if line.strip()]

In [3]:
print(chinese_data[:4])
print(len(chinese_data))

['度门能不访，冒雪屡西东。已想人如玉，遥怜马似骢。乍迷金谷路，稍变上阳宫。还比相思意，纷纷正满空。', '逍遥东城隅，双树寒葱蒨。广庭流华月，高阁凝余霰。杜门非养素，抱疾阻良䜩。孰谓无他人，思君岁云变。官曹亮先忝，陈躅慙俊彥。岂知晨与夜，相代不相见。缄书问所如，詶藻当芬绚。', '川上风雨来，须臾满城阙。岧峣青莲界，萧条孤兴发。前山遽已净，阴霭夜来歇。乔木生夏凉，流云吐华月。严城自有限，一水非难越。相望曙河远，高斋坐超忽。', '庭树忽已暗，故人那不来。祗因厌烦暑，永日坐霜台。']
57586


## 文本切分，并用bert-base-chinese分词&存储

In [4]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
from transformers import AutoTokenizer
import numpy as np
import os
import multiprocessing as mp
from tqdm import tqdm

# 初始化分词器
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese", cache_dir="../model/bert-base-chinese")
tokenizer.add_special_tokens(special_tokens_dict={'eos_token': '<|endoftext|>'})

1

In [5]:
# 添加结束标记 <|endoftext|> 到分词器
tokenizer.add_special_tokens(special_tokens_dict={'eos_token': '<|endoftext|>'})

# 设置每个片段的最大 token 数
context_length = 256

def tokenize(doc):
    # 为每个文档添加 <|endoftext|> 标记
    text_with_eot = doc.replace("\n", "") + tokenizer.eos_token  # 去掉换行符  # 这里会使用已经添加的结束标记
    tokens = tokenizer(
        text_with_eot,
        truncation=True,
        max_length=context_length,
        return_tensors="np"
    )["input_ids"][0][1:-1]

    tokens_np = np.array(tokens.flatten(), dtype=np.uint16)

    assert (0 <= tokens_np).all() and (tokens_np < 2**16).all(), "token 字典对于 uint16 来说太大"
    return tokens_np

def write_datafile(filename, tokens_np):
    np.save(filename, tokens_np)

shard_size = int(1e6)  # 每个分片的 token 数

output_dir = "tang_tokenized_data"
os.makedirs(output_dir, exist_ok=True)

# 假设中文数据是一个列表，每个元素是一个文档文本
chinese_data = chinese_data
# chinese_data = ["文档1的内容", "文档2的内容", "文档3的内容"]  # 示例数据

nprocs = max(1, os.cpu_count() // 2)
# 标记所有文档并写入输出分片，每个分片shard_size令牌（最后一个分片有剩余）
# with mp.Pool(nprocs) as pool: # 多线程
shard_index = 0

# preallocate buffer 以保存当前分片
all_tokens_np = np.empty((shard_size,), dtype=np.uint16)
token_count = 0
progress_bar = None

# 判断当前分片中是否有足够的空间用于新token？
# for tokens in pool.imap(tokenize, chinese_data, chunksize=16):
for token in chinese_data:
    tokens = tokenize(token)
    if token_count + len(tokens) < shard_size:
        # 只需将 Token 附加到当前分片
        all_tokens_np[token_count:token_count + len(tokens)] = tokens
        token_count += len(tokens)

        # 更新进度条
        if progress_bar is None:
            progress_bar = tqdm(total=shard_size, unit="tokens", desc=f"Shard {shard_index}")
        progress_bar.update(len(tokens))
    else:
        # 写入当前分片并启动新分片
        split = "val" if shard_index == 0 else "train"
        filename = os.path.join(output_dir, f"tang_{split}_{shard_index:06d}.npy")


        # 将文档拆分为适合此分片的任何内容，其余的转到下一个
        remainder = shard_size - token_count
        progress_bar.update(remainder)
        all_tokens_np[token_count:token_count + remainder] = tokens[:remainder]
        write_datafile(filename, all_tokens_np)
        shard_index += 1
        progress_bar = None

        # 使用当前文档的剩余部分填充下一个分片
        all_tokens_np[0:len(tokens) - remainder] = tokens[remainder:]
        token_count = len(tokens) - remainder

# 将任何剩余的 Token 写入最后一个分片
if token_count != 0:
    split = "val" if shard_index == 0 else "train"
    filename = os.path.join(output_dir, f"tang_{split}_{shard_index:06d}.npy")
    write_datafile(filename, all_tokens_np[:token_count])

Shard 1:  13%|█▎        | 131206/1000000 [00:00<00:05, 153392.08tokens/s]

KeyboardInterrupt: 

In [5]:
# 观察npy文件
import numpy as np
tokens = np.load("tang_tokenized_data/tang_train_000001.npy") # 1000000
print(len(tokens))

1000000


##  测试bert-base-chinese分词器
<br>

1. 因为GPT是自回归的语言模型，不会停止输出，需要添加终止标志

In [10]:
# data = "侯枉高鉴，举善掩瑕疵。斯民本已安，工拙两无施。何以酬明德，岁晏不磷缁。时节乃来集，欣怀方载驰。平明大府开，一得拜光辉。温如春风至，肃若严霜威。羣属所载瞻，而忘倦与饥。公堂燕华筵，礼罢复言辞。将从平门道，憩车沣水湄。山川降嘉。"
data1 = """前年东秦夏六月，望日拜恩初赐玦。降秩削职迁黄州，仓皇束装三日发。故国东平不入城，北山一夜辞松栝。亲朋问讯若梦寐，骨肉分留作胡越。论罪岂合有民社，抵谳正欲加𫓧钺。仇家之议不尽用，天地寛仁日月察。才令分司置之蕲，上表谢恩秋已末。以御魑魅乃其分，欲居蛮夷圣犹屑。蕲也虽僻自善地，回环山溪富林樾。平生雅志在江湖，颇与蕲人相缔结。收拾孥属已团聚，南北无心更分别。白鱼煮玉秔炊珠，佐以秋菘与春蕨。筑室求田虽未就，典衣卖装略无阙。大谬不然心已忘，笑人非工已愈拙。人生端若梦栩栩，事去何庸书咄咄。大儿调邑换江南，要虽分房无远别。地劣两舍不宿舂，晨起为书午可达。邑四万户号难治，民杂江闽吏贪猾。锄彊洗恶勿著意，鱼逃至清人忌洁。化以诚心磨以久，教而后刑不怨杀。得闻无毁亦无誉，以慰萧萧双白髪。<|endoftext|>
"""
data = "补吏多下迁，罢归聊自度。园庐既芜没，烟景空澹泊。闲居养疴瘵，守素甘葵藿。颜鬓日衰耗，冠带亦寥落。青苔已生路，绿筠始分箨。夕气下遥阴，微风动疎薄。草玄良见诮，杜门无请讬。非君好事者，谁来顾寂寞。"
# tokenizer.add_special_tokens(special_tokens_dict={'eos_token': '<|endoftext|>'})
print(1)
outputs = tokenizer(
    data + tokenizer.eos_token,
    truncation=True,
    # max_length=50,
    return_overflowing_tokens=True,
    return_length=True,
)


1


In [11]:
# 可以发现实际上对诗词进行tokenization时，token数和文本数基本是等长的，编码 21128 是 <|endoftext|>
# print(outputs)
print(outputs["input_ids"][0][1:-1])
print(len(outputs["input_ids"][0][1:-1]))
print(len(data)+1)  # <|endoftext|>: 1
# print(outputs["attention_mask"])
# print(outputs["token_type_ids"])

[6133, 1401, 1914, 678, 6810, 8024, 5387, 2495, 5464, 5632, 2428, 511, 1736, 2416, 3188, 5697, 3766, 8024, 4170, 3250, 4958, 4079, 3788, 511, 7312, 2233, 1075, 100, 100, 8024, 2127, 5162, 4491, 5878, 5977, 511, 7582, 7779, 3189, 6139, 5450, 8024, 1094, 2372, 771, 2178, 5862, 511, 7471, 5726, 2347, 4495, 6662, 8024, 5344, 5035, 1993, 1146, 100, 511, 1911, 3698, 678, 6898, 7346, 8024, 2544, 7599, 1220, 100, 5946, 511, 5770, 4371, 5679, 6224, 100, 8024, 3336, 7305, 3187, 6435, 100, 511, 7478, 1409, 1962, 752, 5442, 8024, 6443, 3341, 7560, 2163, 2174, 511, 21128]
97
97


In [10]:
# 可以发现实际上对诗词进行tokenization时，token数和文本数基本是等长的，编码 21128 是 <|endoftext|>
# print(outputs)
print(outputs["input_ids"][0][1:-1])
print(len(data)+1)  # <|endoftext|>: 1
# print(outputs["attention_mask"])
# print(outputs["token_type_ids"])

[6133, 1401, 1914, 678, 6810, 8024, 5387, 2495, 5464, 5632, 2428, 511, 1736, 2416, 3188, 5697, 3766, 8024, 4170, 3250, 4958, 4079, 3788, 511, 7312, 2233, 1075, 100, 100, 8024, 2127, 5162, 4491, 5878, 5977, 511, 7582, 7779, 3189, 6139, 5450, 8024, 1094, 2372, 771, 2178, 5862, 511, 7471, 5726, 2347, 4495, 6662, 8024, 5344, 5035, 1993, 1146, 100, 511, 1911, 3698, 678, 6898, 7346, 8024, 2544, 7599, 1220, 100, 5946, 511, 5770, 4371, 5679, 6224, 100, 8024, 3336, 7305, 3187, 6435, 100, 511, 7478, 1409, 1962, 752, 5442, 8024, 6443, 3341, 7560, 2163, 2174, 511, 21128]
97


In [ ]:
# 解码测试
text = tokenizer.decode(outputs["input_ids"][0], skip_special_tokens=True)
print(outputs["input_ids"][0])
# 打印解码后的文本
print(text)

In [ ]:
tokenizer.encode("谁 来 顾 寂 寞")

In [ ]:
test = tokenizer(
        data,
        truncation=True,
        max_length=context_length,
        return_tensors="np"
    )["input_ids"]
test